In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
# Travel agent to find flights to and from your ideal destination

client = MultiServerMCPClient({
    "travel_server": {
        "transport": "streamable_http",
        "url": "https://mcp.kiwi.com"
    }
})


In [3]:
# get tools
tools = await client.get_tools()
print(tools)

[StructuredTool(name='search-flight', description='\n# Search for a flight\n\n## Description\n\nUses the Kiwi API to search for available flights between two locations on a specific date.\n\n## How it works\n\nThe tool will:\n1. Search for matching locations to resolve airport codes\n2. Find available flights for the specified route and date range\n\n## Method\n\nCall this tool whenever a user wants to search for flights, regardless of whether they provided exact airport codes or just city names.\n\nYou should display the returned results in a markdown table format: Group the results by price (those who are the cheapest), duration (those who are the shortest, i.e. have the smallest \'totalDurationInSeconds\') and the rest (those that could still be interesting).\n\nAlways display for each flight in order:\n  - In the 1st column: The departure and arrival airports, including layovers (e.g. "Paris CDG → Barcelona BCN → Lisbon LIS")\n  - In the 2nd column: The departure and arrival dates 

In [4]:
from langchain.tools import tool
from tavily import TavilyClient
from typing import Dict, Any

tavily_client = TavilyClient()

# Venue agent that searches the web for a wedding venue
@tool
def venue_web_search(query: str) -> Dict[str, Any]:
    """ Performs web search for information """
    
    return tavily_client.search(query)


In [5]:
# DJ agent that scours a music database for a playlist, matching the genre
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(
    "sqlite:////Users/koushalsmodi/Desktop/MachineLearning/MachineLearningProjects/LangGraph/lca-lc-foundations/notebooks/module-2/resources/Chinook.db"
)

@tool
def query_playlist_db(query: str) -> str:

    """Query the database for playlist information"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error querying database: {e}"
    


In [6]:
from langchain.agents import AgentState

class WeddingState(AgentState):
    origin: str 
    destination: str
    guest_count: str 
    genre: str

In [13]:
from langchain.agents import create_agent

travel_agent = create_agent(
    model = "gpt-5-nano",
    tools = tools,
    system_prompt = """ 
    You are a travel agent. Search for flights to the desired destination wedding location from the origin. No need to ask any follow up questions.
    Find the best flight options based on the following criteria:
    - Price (lowest, economy class)
    - Duration (shortest)
    - Date (time of year which you believe is best for a wedding at this location)
    To make things easy, only look for one ticket, one way.
    You may need to make multiple searches to iteratively find the best options.
    You will be given no extra information, only the origin and destination. It is your job to think critically about the best options.
    Once you have found the best options, let the user know your shortlist of options.
    """
)

venue_agent = create_agent(
    model = "gpt-5-nano",
    tools = [venue_web_search],
    system_prompt = """ 
    You specialize in venue suggestions, especially for weddings. Based on the desired location, search for venues and consider the desired capacity while searching.
    No need to ask any follow up questions. Use the following criteria:
    - Price (lowest)
    - Capacity(exact or 10% higher match)
    - Reviews (highest)
    - Location closer to airport.
    You may need to make multiple searches to iteratively find the best options.
    """
)

playlist_agent = create_agent(
    model = "gpt-5-nano",
    tools = [query_playlist_db],
    system_prompt = """ 
    You are a DJ playlist specialist. Query the sql database and curate the perfect playlist for a wedding given a genre.
    Once you have your playlist, calculate the total duration and cost of the playlist, each song has an associated price.
    Do not ask questions.
    Do not come back empty handed, keep trying to query the db until you find a list of songs.
    You may need to make multiple queries to iteratively find the best options. 
    """
)

In [9]:
from langchain.tools import ToolRuntime
from langchain.messages import HumanMessage, ToolMessage
from langgraph.types import Command

@tool
async def search_flights(runtime: ToolRuntime) -> str:
    """Travel agent searches for flights to the desired destination wedding location."""
    origin = runtime.state["origin"]
    destination = runtime.state["destination"]
    response = await travel_agent.ainvoke({"messages": [HumanMessage(content=f"Find flights from {origin} to {destination}")]})
    return response['messages'][-1].content

@tool
def search_venues(runtime: ToolRuntime) -> str:
    """Venue agent chooses the best venue for the given location and capacity."""
    destination = runtime.state["destination"]
    capacity = runtime.state["guest_count"]
    query = f"Find wedding venues in {destination} for {capacity} guests"
    response = venue_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def suggest_playlist(runtime: ToolRuntime) -> str:
    """Playlist agent curates the perfect playlist for the given genre."""
    genre = runtime.state["genre"]
    query = f"Find {genre} tracks for wedding playlist"
    response = playlist_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def update_state(origin: str, destination: str, guest_count: str, genre: str, runtime: ToolRuntime) -> str:
    """Update the state when you know all of the values: origin, destination, guest_count, genre"""
    return Command(update={
        "origin": origin, 
        "destination": destination, 
        "guest_count": guest_count, 
        "genre": genre, 
        "messages": [ToolMessage("Successfully updated state", tool_call_id=runtime.tool_call_id)]}
        )

In [14]:
from langchain.agents import create_agent

coordinator = create_agent(
    model="gpt-5-nano",
    tools=[search_flights, search_venues, suggest_playlist, update_state],
    state_schema=WeddingState,
    system_prompt="""
    You are a wedding coordinator. Delegate tasks to your specialists for flights, venues and playlists.
    First find all the information you need to update the state. Once that is done you can delegate the tasks.
    Once you have received their answers, coordinate the perfect wedding for me.
    """
)

In [15]:
from langchain.messages import HumanMessage

response = await coordinator.ainvoke(
    {
        "messages": [HumanMessage(content="I'm from Bharuch, Gujarat, India and I'd like a wedding in Mumbai for 240 guests, bollywood")],
    }
)

from pprint import pprint
pprint(response)

{'destination': 'Mumbai, Maharashtra, India',
 'genre': 'Bollywood',
 'guest_count': '240',
 'messages': [HumanMessage(content="I'm from Bharuch, Gujarat, India and I'd like a wedding in Mumbai for 240 guests, bollywood", additional_kwargs={}, response_metadata={}, id='bdf99f20-72c6-4a3a-bcb5-0c566feaf78c'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1007, 'prompt_tokens': 297, 'total_tokens': 1304, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 960, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CpRX6qZgRicdljrGEHpoz0e7Owdxu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b443f-f2af-7a83-867c-2293fd4bf736-0', tool_calls=[{'name': 'update_s

In [16]:
print(response["messages"][-1].content)


Fantastic—we’ve started pulling together the pieces for a Bollywood-inspired Mumbai wedding for 240 guests. Here’s what my specialists have gathered, plus some recommended next steps.

Flight options (origin near Bharuch; closest practical airport is Surat, STV)
- Cheapest 1-stop options (STV → BOM via DEL)
  - Dep 29/12/2025 22:25 → Arr 30/12/2025 05:35; 7h10m; INR 12,812
    Booking: https://on.kiwi.com/55qYll
  - Dep 29/12/2025 21:10 → Arr 30/12/2025 05:35; 8h25m; INR 12,812
    Booking: https://on.kiwi.com/BT9QTm
  - Dep 29/12/2025 22:25 → Arr 30/12/2025 07:50; 9h25m; INR 12,812
    Booking: https://on.kiwi.com/kbtjcR
- Fastest option (STV → BOM via BLR)
  - Dep 29/12/2025 17:15 → Arr 29/12/2025 23:35; ~5h20m; INR 15,577
    Booking: https://on.kiwi.com/MMbedv
- Other mid-range options (via DEL, DEL + other times)
  - INR ~14,088 – 14,445, durations ~6–6.5h
    Booking examples: https://on.kiwi.com/EuiuSQ, https://on.kiwi.com/g3oLGf

Note
- Bharuch itself doesn’t have a big commerc